In [179]:
import numpy as np
import random
from random import randint
import gmpy2
from scipy.stats import chi2
import math
import time

Допоміжні ф-ї

In [180]:
def conv_to_list(n, t):
    num = [int(i) for i in bin(n)[2:]]
    while len(num) < t:
        num.insert(0, 0)
    return num

def display(lst):
    for i in lst:
        print(i)
        
def bin_lst_to_int(lst):
    return int("".join(str(x) for x in lst), 2)

def gen_bits(N):
    d = np.random.randint(0, 2, N).tolist()
    d_int = bin_lst_to_int(d)
    if d_int == 0: #check if all elements in d are 0
        while d_int == 0:
            d = np.random.randint(0, 2, N).tolist()
            d_int = bin_lst_to_int(d)
            print(d)
    return d

def hex_to_list(hex, b):
    n = int(hex, 16)
    number = []
    while n > 0:
        number.insert(0, n % b)  # take remainder of division and add it leftmost of the array
        n = n // b
    return number

def hex_to_int(hex):
    return bin_lst_to_int(hex_to_list(hex, 2))

def bits_to_bytes(lst): # конвертує згенеровану бітову посл-ть в байтову
    res = []
    chunks = [lst[i:i + 8] for i in range(0, len(lst), 8)] 
    for item in chunks:
        res.append(bin_lst_to_int(item))
    return res

1. Вбудований генератор псевдовипадкових чисел

In [300]:
def rand(N):
    random.seed()
    res = []
    for i in range(0,N):
        if i%25000 == 0: print(i)
        num = randint(0, 255)
        res.append(num)
    return res

2. Генератор LehmerLow

In [299]:
m = 2**32
a = 2**16 + 1
c = 119

def LehmerLow(N):
    prev = np.random.randint(0,2**31) 
    res = []
    for i in range(N):
        if i%25000 == 0: print(i)
        x_i = (a * prev + c)%m
        res_i = conv_to_list(x_i, 32)[-8:]
        res_i = bin_lst_to_int(res_i)
        res.append(res_i)
        prev = x_i
    return res    

3. Генератор LehmerHigh

In [298]:
m = 2**32
a = 2**16 + 1
c = 119

def LehmerHigh(N):
    prev = np.random.randint(0,2**31) 
    res = []
    for i in range(N):
        if i%25000 == 0: print(i)
        x_i = (a * prev + c)%m
        res_i = conv_to_list(x_i, 32)[:8]
        res_i = bin_lst_to_int(res_i)
        res.append(res_i)
        prev = x_i
    return res    

4. Генератор псевдовипадкових двійкових послідовностей L20

In [297]:
def L20(N):
    d1 = np.random.randint(0, 2, 20)
    d = d1.tolist()
    for i in range(20, 20 + N): 
        if i%200000 == 0: print(i)
        x = d[i-3] ^ d [i-5] ^ d[i-9] ^ d[i-20]
        d.append(x)  
    return d

5. Генератор псевдовипадкових двійкових послідовностей L89

In [296]:
def L89(N):
    d = gen_bits(89)
    for i in range(89, 89 + N): 
        if i%100000 == 0: print(i)
        x = d[i-38] ^ d [i-89]
        d.append(x)
    return d

6. Генератор Джиффі

In [295]:
def L11(n):
    d1 = np.random.randint(0, 2, 11)
    d = d1.tolist()
    for i in range(11, n+1): 
        x = d[i-11] ^ d [i-9]
        d.append(x)
    return d

def L9(n):
    d1 = np.random.randint(0, 2, 9)
    d = d1.tolist()
    for i in range(9, n+1): 
        x = d[i-9] ^ d [i-8] ^ d[i-6] ^ d[i-5]
        d.append(x)
    return d

def L10(n):
    d1 = np.random.randint(0, 2, 10)
    d = d1.tolist()
    for i in range(10, n+1): 
        x = d[i-10] ^ d [i-7]
        d.append(x)
    return d

def Geffe(n):
    d = []
    l11 = L11(n)
    l9 = L9(n)
    l10 = L10(n)
    for i in range(0,n):
        if i%100000 == 0: print(i)
        x = l11[i]
        y = l9[i]
        s = l10[i]
        #z = (s * x) ^ ((1 ^ s) * y)
        if s == 1: z = x
        elif s == 0: z = y
        else: print('error, s = ', s)
        d.append(z)
    return d

7. Генератор Вольфрама

In [294]:
def Wolfram(N):
    r = gen_bits(32)
    res = []
    for i in range(N):
        if i%100000 == 0: print(i)
        x_i = r[-1]
        r = np.roll(r, -1) ^ (r | np.roll(r, 1)) 
        res.append(x_i)
    return res

8. Генератор "Бібліотекар"

In [293]:
def Librarian():
    f = open(r'The_Goldfinch_ru1.txt', 'r', encoding='utf8', errors='ignore')
    text = f.read()
    f.close()
    b = text.encode('utf-8')
    gen_nums = []
    for i in range(0,len(b)): gen_nums.append(b[i])
    return gen_nums

9. Генератор Блюма-Мікалі (BM)

In [292]:
P = "CEA42B987C44FA642D80AD9F51F10457690DEF10C83D0BC1BCEE12FC3B6093E3"
A = "5B88C41246790891C095E2878880342E88C79974303BD0400B090FE38A688356"
Q = "675215CC3E227D3216C056CFA8F8822BB486F788641E85E0DE77097E1DB049F1"

def BM(N):
    p = hex_to_int(P)
    a = hex_to_int(A)
    q = hex_to_int(Q)
    T = random.randint(0,p-1)
    res = []
    for i in range(N):
        if i%100000 == 0: print(i)
        if T < gmpy2.c_div(p-1, 2):
            x = 1
        else:
            x = 0
        res.append(x)
        T = gmpy2.powmod(a,T,p)
    return res

10. Генератор BM_bytes (байтовий)

In [291]:
P = "CEA42B987C44FA642D80AD9F51F10457690DEF10C83D0BC1BCEE12FC3B6093E3"
A = "5B88C41246790891C095E2878880342E88C79974303BD0400B090FE38A688356"
Q = "675215CC3E227D3216C056CFA8F8822BB486F788641E85E0DE77097E1DB049F1"

def BM_bytes(N):
    p = hex_to_int(P)
    a = hex_to_int(A)
    q = hex_to_int(Q)
    T = random.randint(0,p-1)
    res = []
    for i in range(N):
        if i%25000 == 0: print(i)
        g = gmpy2.c_div(gmpy2.mul(256,T), p-1)
        for k in range(g-1, g):
            if k < g and k >= g-1:
                res.append(k)
                break
        T = gmpy2.powmod(a,T,p)
    return res

11. Генератор BBS (бітовий)

In [288]:
p = 'D5BBB96D30086EC484EBA3D7F9CAEB07'
q = '425D2B9BFDB25B9CF6C416CC6E37B59C1F'
n = int(p,16) * int(q,16)

def BBS_bin(N):
    
    r = [randint(0,n)]
    x = []
    for i in range (1,N):
        if i%100000 == 0: print(i)
        R = (r[i-1] ** 2) % n
        r.append(R)
        x.append(R % 2)
    return x

12. Генератор BBS (байтовий)

In [301]:
def BBS_b(N):
    
    r = [randint(0,n)]
    x = []
    for i in range (1,N):
        if i%25000 == 0: print(i)
        R = (r[i-1] ** 2) % n
        r.append(R)
        x.append(R % 256)
    return x

## Тести

In [193]:
alphas = [0.01, 0.05, 0.1]
quantiles = [2.326347874, 1.644853627, 1.281551566]

def hi2_check(hi,l):
    print("hi^2=", hi)
    for i in range (0,3):
        hi_lim = math.sqrt(2*l) * quantiles[i] + l
        print(f"hi^2 (1-{alphas[i]})=", hi_lim)
        if hi <= hi_lim: 
            print('Test passed for alpha = ', alphas[i])
        else: 
            print('Test failed for alpha = ', alphas[i])

 Рівноймовірність:

In [194]:
def eq_prob(lst):
    i = 0
    freqs = []
    n = len(lst)
    for i in range (0,256):
        count = 0
        for j in range (0,n): 
            if lst[j] == i: count = count+1
        freqs.append(count)
    n_j = n/256
    hi_2 = 0
    for i in range(0,256): hi_2 = hi_2 + ((freqs[i]-n_j) ** 2) / n_j
    hi2_check(hi_2,255)

Незалежність

In [204]:
def independence(lst):
    n = len(lst)//2
    pairs = [(lst[2*i],lst[2*i+1]) for i in range (0,n)]
    pairs_dict = dict()
    pairs_unique = list(set(pairs))
    for item in pairs_unique:
        num = pairs.count(item)
        pairs_dict.update({item: num})
    freqs = np.zeros((256, 256))
    for item in pairs_dict:
        freqs[item[0]][item[1]] = pairs_dict[item]
    
    sum_1 = [] # сколько раз i-й на 1 месте
    for i in range (0,256):
        sum_1.append(sum(freqs[i][j] for j in range(0,255)))
    sum_2 = [] # сколько раз j-й на 2 месте
    for j in range (0,256):
        sum_2.append(sum(freqs[i][j] for i in range(0,255)))
    
    hi_2 = 0
    for i in range(0,256):
        for j in range(0,256):
            if sum_1[i] * sum_2[j] != 0: hi_2 = hi_2 + (freqs[i][j]** 2)/ (sum_1[i] * sum_2[j])     
    hi_2 = n * (hi_2-1)
    hi2_check(hi_2,255 ** 2)

Однорідність:

In [209]:
def homogeneity(lst):
    m = len(lst)
    r = int(math.log(m, 2))
    print("m=",m, "r=",r)
    m1 = m // r
    n = m1 * r
    
    chunks = [lst[i:i + m1] for i in range(0, n, m1)] 
    
    appear = np.zeros((256, r)) # к-ть появ і_го байта в j_му відрізку
    for i in range(256):
        for j in range(r):
            appear[i][j] = chunks[j].count(i)
    
    sum_1 = []
    for i in range(256):
        v_i = np.sum(appear[i])
        sum_1.append(v_i)
        
 
    al_j = m1
    
    hi_2 = 0
    for i in range(256):
        for j in range(r):
            if sum_1[i] != 0:
                hi_2 = hi_2 + (appear[i][j] ** 2)/(sum_1[i] * al_j)
    hi_2 = n * (hi_2-1)
    hi2_check(hi_2, 255 * (r - 1))   

In [200]:
def three_tests(lst):
    print("---- TEST 1 ------")
    eq_prob(lst)
    print("---- TEST 2 ------")
    independence(lst)
    print("---- TEST 3 ------")
    homogeneity(lst)

## Застосування тестів

In [303]:
def to_file(lst,name): #ф-я щоб записати згенеровану посл-ть у файл
    file = open(f"{name}.txt", "w")
    for element in lst:
        file.write(str(element) + ",")
    file.close()
    
def from_file(filename): #ф-я щоб дістати з файлу (не знадобилось)
    f = open(filename, "r")
    content = f.read()
    lst = content.split(",")
    f.close()

In [305]:
n = 125000  #для байтових генераторів
m = 1000000 #для бітових генераторів

In [306]:
start = time.time()
python_gen = rand(n)
end = time.time()
print("python_gen time =", end - start)

# to_file(python_gen, "python_gen")

print("Вбудований генератор псевдовипадкових чисел:")
three_tests(python_gen)

0
25000
50000
75000
100000
python_gen time = 0.1958169937133789
Вбудований генератор псевдовипадкових чисел:
---- TEST 1 ------
hi^2= 245.30483200000015
hi^2 (1-0.01)= 307.53633180705333
Test passed for alpha =  0.01
hi^2 (1-0.05)= 292.1460248434483
Test passed for alpha =  0.05
hi^2 (1-0.1)= 283.9415091576389
Test passed for alpha =  0.1
---- TEST 2 ------
hi^2= 65334.34331198118
hi^2 (1-0.01)= 65863.9379421232
Test passed for alpha =  0.01
hi^2 (1-0.05)= 65618.1744483926
Test passed for alpha =  0.05
hi^2 (1-0.1)= 65487.158839407006
Test passed for alpha =  0.1
---- TEST 3 ------
m= 125000 r= 16
hi^2= 3919.281328193674
hi^2 (1-0.01)= 4028.4723381595445
Test passed for alpha =  0.01
hi^2 (1-0.05)= 3968.8659355964824
Test passed for alpha =  0.05
hi^2 (1-0.1)= 3937.089982981645
Test passed for alpha =  0.1


In [307]:
start = time.time()
lehmer_low = LehmerLow(n)
end = time.time()
print("lehmer_low time =", end - start)

# to_file(lehmer_low, "lehmer_low")

print("генератор LehmerLow:")
three_tests(lehmer_low)

0
25000
50000
75000
100000
lehmer_low time = 1.4129748344421387
генератор LehmerLow:
---- TEST 1 ------
hi^2= 85.05702400000001
hi^2 (1-0.01)= 307.53633180705333
Test passed for alpha =  0.01
hi^2 (1-0.05)= 292.1460248434483
Test passed for alpha =  0.05
hi^2 (1-0.1)= 283.9415091576389
Test passed for alpha =  0.1
---- TEST 2 ------
hi^2= 1961284.7414280763
hi^2 (1-0.01)= 65863.9379421232
Test failed for alpha =  0.01
hi^2 (1-0.05)= 65618.1744483926
Test failed for alpha =  0.05
hi^2 (1-0.1)= 65487.158839407006
Test failed for alpha =  0.1
---- TEST 3 ------
m= 125000 r= 16
hi^2= 3016.7168653813765
hi^2 (1-0.01)= 4028.4723381595445
Test passed for alpha =  0.01
hi^2 (1-0.05)= 3968.8659355964824
Test passed for alpha =  0.05
hi^2 (1-0.1)= 3937.089982981645
Test passed for alpha =  0.1


In [310]:
start = time.time()
lehmer_high = LehmerHigh(125000)
end = time.time()
print("lehmer_high time =", end - start)

# to_file(lehmer_high, "lehmer_high")

print("генератор LehmerHigh:")
three_tests(lehmer_high)

0
25000
50000
75000
100000
lehmer_high time = 1.3133463859558105
генератор LehmerHigh:
---- TEST 1 ------
hi^2= 31875000.0
hi^2 (1-0.01)= 307.53633180705333
Test failed for alpha =  0.01
hi^2 (1-0.05)= 292.1460248434483
Test failed for alpha =  0.05
hi^2 (1-0.1)= 283.9415091576389
Test failed for alpha =  0.1
---- TEST 2 ------
hi^2= 0.0
hi^2 (1-0.01)= 65863.9379421232
Test passed for alpha =  0.01
hi^2 (1-0.05)= 65618.1744483926
Test passed for alpha =  0.05
hi^2 (1-0.1)= 65487.158839407006
Test passed for alpha =  0.1
---- TEST 3 ------
m= 125000 r= 16
hi^2= 0.0
hi^2 (1-0.01)= 4028.4723381595445
Test passed for alpha =  0.01
hi^2 (1-0.05)= 3968.8659355964824
Test passed for alpha =  0.05
hi^2 (1-0.1)= 3937.089982981645
Test passed for alpha =  0.1


In [311]:
start = time.time()
l20 = bits_to_bytes(L20(m))
end = time.time()
print("l20 time =", end - start)

# to_file(l20, "l20")

print("Генератор L20:")
three_tests(l20)

200000
400000
600000
800000
1000000
l20 time = 1.815323829650879
Генератор L20:
---- TEST 1 ------
hi^2= 246.17398782429208
hi^2 (1-0.01)= 307.53633180705333
Test passed for alpha =  0.01
hi^2 (1-0.05)= 292.1460248434483
Test passed for alpha =  0.05
hi^2 (1-0.1)= 283.9415091576389
Test passed for alpha =  0.1
---- TEST 2 ------
hi^2= 62160.561591003345
hi^2 (1-0.01)= 65863.9379421232
Test passed for alpha =  0.01
hi^2 (1-0.05)= 65618.1744483926
Test passed for alpha =  0.05
hi^2 (1-0.1)= 65487.158839407006
Test passed for alpha =  0.1
---- TEST 3 ------
m= 125003 r= 16
hi^2= 3852.106317403468
hi^2 (1-0.01)= 4028.4723381595445
Test passed for alpha =  0.01
hi^2 (1-0.05)= 3968.8659355964824
Test passed for alpha =  0.05
hi^2 (1-0.1)= 3937.089982981645
Test passed for alpha =  0.1


In [312]:
start = time.time()
l89 = bits_to_bytes(L89(m))
end = time.time()
print("l89 time =", end - start)

# to_file(l89, "l89")

print("Генератор L89:")
three_tests(l89)

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
l89 time = 0.9397451877593994
Генератор L89:
---- TEST 1 ------
hi^2= 263.21735513390706
hi^2 (1-0.01)= 307.53633180705333
Test passed for alpha =  0.01
hi^2 (1-0.05)= 292.1460248434483
Test passed for alpha =  0.05
hi^2 (1-0.1)= 283.9415091576389
Test passed for alpha =  0.1
---- TEST 2 ------
hi^2= 66492.45203717746
hi^2 (1-0.01)= 65863.9379421232
Test failed for alpha =  0.01
hi^2 (1-0.05)= 65618.1744483926
Test failed for alpha =  0.05
hi^2 (1-0.1)= 65487.158839407006
Test failed for alpha =  0.1
---- TEST 3 ------
m= 125012 r= 16
hi^2= 3723.49537977719
hi^2 (1-0.01)= 4028.4723381595445
Test passed for alpha =  0.01
hi^2 (1-0.05)= 3968.8659355964824
Test passed for alpha =  0.05
hi^2 (1-0.1)= 3937.089982981645
Test passed for alpha =  0.1


In [313]:
start = time.time()
geffe = bits_to_bytes(Geffe(m))
end = time.time()
print("geffe time =", end - start)

# to_file(geffe, "geffe")

print("Генератор Джиффі:")
three_tests(geffe)

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
geffe time = 2.173771381378174
Генератор Джиффі:
---- TEST 1 ------
hi^2= 233.1806720000001
hi^2 (1-0.01)= 307.53633180705333
Test passed for alpha =  0.01
hi^2 (1-0.05)= 292.1460248434483
Test passed for alpha =  0.05
hi^2 (1-0.1)= 283.9415091576389
Test passed for alpha =  0.1
---- TEST 2 ------
hi^2= 73961.93883616145
hi^2 (1-0.01)= 65863.9379421232
Test failed for alpha =  0.01
hi^2 (1-0.05)= 65618.1744483926
Test failed for alpha =  0.05
hi^2 (1-0.1)= 65487.158839407006
Test failed for alpha =  0.1
---- TEST 3 ------
m= 125000 r= 16
hi^2= 3552.471171690401
hi^2 (1-0.01)= 4028.4723381595445
Test passed for alpha =  0.01
hi^2 (1-0.05)= 3968.8659355964824
Test passed for alpha =  0.05
hi^2 (1-0.1)= 3937.089982981645
Test passed for alpha =  0.1


In [314]:
start = time.time()
wolfram = bits_to_bytes(Wolfram(m))
end = time.time()
print("wolfram time =", end - start)

# to_file(wolfram, "wolfram")

print("Генератор Вольфрама:")
three_tests(wolfram)

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
wolfram time = 43.3988618850708
Генератор Вольфрама:
---- TEST 1 ------
hi^2= 269.1558400000001
hi^2 (1-0.01)= 307.53633180705333
Test passed for alpha =  0.01
hi^2 (1-0.05)= 292.1460248434483
Test passed for alpha =  0.05
hi^2 (1-0.1)= 283.9415091576389
Test passed for alpha =  0.1
---- TEST 2 ------
hi^2= 66303.73511491033
hi^2 (1-0.01)= 65863.9379421232
Test failed for alpha =  0.01
hi^2 (1-0.05)= 65618.1744483926
Test failed for alpha =  0.05
hi^2 (1-0.1)= 65487.158839407006
Test failed for alpha =  0.1
---- TEST 3 ------
m= 125000 r= 16
hi^2= 3851.1002734807385
hi^2 (1-0.01)= 4028.4723381595445
Test passed for alpha =  0.01
hi^2 (1-0.05)= 3968.8659355964824
Test passed for alpha =  0.05
hi^2 (1-0.1)= 3937.089982981645
Test passed for alpha =  0.1


In [315]:
start = time.time()
librarian = Librarian()[:n]
end = time.time()
print("librarian time =", end - start)

# to_file(librarian, "librarian")

print("Генератор «Бібліотекар»:")
three_tests(librarian)

librarian time = 0.2000415325164795
Генератор «Бібліотекар»:
---- TEST 1 ------
hi^2= 4067291.4754559994
hi^2 (1-0.01)= 307.53633180705333
Test failed for alpha =  0.01
hi^2 (1-0.05)= 292.1460248434483
Test failed for alpha =  0.05
hi^2 (1-0.1)= 283.9415091576389
Test failed for alpha =  0.1
---- TEST 2 ------
hi^2= 877532.2087601556
hi^2 (1-0.01)= 65863.9379421232
Test failed for alpha =  0.01
hi^2 (1-0.05)= 65618.1744483926
Test failed for alpha =  0.05
hi^2 (1-0.1)= 65487.158839407006
Test failed for alpha =  0.1
---- TEST 3 ------
m= 125000 r= 16
hi^2= 3203.676942960109
hi^2 (1-0.01)= 4028.4723381595445
Test passed for alpha =  0.01
hi^2 (1-0.05)= 3968.8659355964824
Test passed for alpha =  0.05
hi^2 (1-0.1)= 3937.089982981645
Test passed for alpha =  0.1


In [316]:
start = time.time()
blum_micali = bits_to_bytes(BM(m))
end = time.time()
print("blum_micali time =", end - start)

# to_file(blum_micali, "blum_micali")

print("Генератор Блюма-Мікалі BM:")
three_tests(blum_micali)

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
blum_micali time = 54.449491024017334
Генератор Блюма-Мікалі BM:
---- TEST 1 ------
hi^2= 269.65145600000017
hi^2 (1-0.01)= 307.53633180705333
Test passed for alpha =  0.01
hi^2 (1-0.05)= 292.1460248434483
Test passed for alpha =  0.05
hi^2 (1-0.1)= 283.9415091576389
Test passed for alpha =  0.1
---- TEST 2 ------
hi^2= 66436.62527499083
hi^2 (1-0.01)= 65863.9379421232
Test failed for alpha =  0.01
hi^2 (1-0.05)= 65618.1744483926
Test failed for alpha =  0.05
hi^2 (1-0.1)= 65487.158839407006
Test failed for alpha =  0.1
---- TEST 3 ------
m= 125000 r= 16
hi^2= 3837.1314065270817
hi^2 (1-0.01)= 4028.4723381595445
Test passed for alpha =  0.01
hi^2 (1-0.05)= 3968.8659355964824
Test passed for alpha =  0.05
hi^2 (1-0.1)= 3937.089982981645
Test passed for alpha =  0.1


In [317]:
start = time.time()
bm_bytes = BM_bytes(n)
end = time.time()
print("bm_bytes time =", end - start)

# to_file(bm_bytes, "bm_bytes")

print("Генератор BM_bytes:")
three_tests(bm_bytes)

0
25000
50000
75000
100000
bm_bytes time = 6.7931435108184814
Генератор BM_bytes:
---- TEST 1 ------
hi^2= 232.83660800000024
hi^2 (1-0.01)= 307.53633180705333
Test passed for alpha =  0.01
hi^2 (1-0.05)= 292.1460248434483
Test passed for alpha =  0.05
hi^2 (1-0.1)= 283.9415091576389
Test passed for alpha =  0.1
---- TEST 2 ------
hi^2= 65994.98676590668
hi^2 (1-0.01)= 65863.9379421232
Test failed for alpha =  0.01
hi^2 (1-0.05)= 65618.1744483926
Test failed for alpha =  0.05
hi^2 (1-0.1)= 65487.158839407006
Test failed for alpha =  0.1
---- TEST 3 ------
m= 125000 r= 16
hi^2= 3825.027286582063
hi^2 (1-0.01)= 4028.4723381595445
Test passed for alpha =  0.01
hi^2 (1-0.05)= 3968.8659355964824
Test passed for alpha =  0.05
hi^2 (1-0.1)= 3937.089982981645
Test passed for alpha =  0.1


In [318]:
start = time.time()
bbs_bin = bits_to_bytes(BBS_bin(m))
end = time.time()
print("bbs_bin time =", end - start)

# to_file(bbs_bin, "bbs_bin")

print("Генератор BBS:")
three_tests(bbs_bin)

100000
200000
300000
400000
500000
600000
700000
800000
900000
bbs_bin time = 1.283982276916504
Генератор BBS:
---- TEST 1 ------
hi^2= 31874488.004096
hi^2 (1-0.01)= 307.53633180705333
Test failed for alpha =  0.01
hi^2 (1-0.05)= 292.1460248434483
Test failed for alpha =  0.05
hi^2 (1-0.1)= 283.9415091576389
Test failed for alpha =  0.1
---- TEST 2 ------
hi^2= -62500
hi^2 (1-0.01)= 65863.9379421232
Test passed for alpha =  0.01
hi^2 (1-0.05)= 65618.1744483926
Test passed for alpha =  0.05
hi^2 (1-0.1)= 65487.158839407006
Test passed for alpha =  0.1
---- TEST 3 ------
m= 125000 r= 16
hi^2= 0.0
hi^2 (1-0.01)= 4028.4723381595445
Test passed for alpha =  0.01
hi^2 (1-0.05)= 3968.8659355964824
Test passed for alpha =  0.05
hi^2 (1-0.1)= 3937.089982981645
Test passed for alpha =  0.1


In [319]:
start = time.time()
bbs_byte = BBS_b(n)
end = time.time()
print("bbs_byte time =", end - start)

# to_file(bbs_byte, "bbs_byte")

print("Генератор BBS_bytes:")
three_tests(bbs_byte)

25000
50000
75000
100000
bbs_byte time = 0.08355569839477539
Генератор BBS_bytes:
---- TEST 1 ------
hi^2= 31873721.012288097
hi^2 (1-0.01)= 307.53633180705333
Test failed for alpha =  0.01
hi^2 (1-0.05)= 292.1460248434483
Test failed for alpha =  0.05
hi^2 (1-0.1)= 283.9415091576389
Test failed for alpha =  0.1
---- TEST 2 ------
hi^2= 62499.0
hi^2 (1-0.01)= 65863.9379421232
Test passed for alpha =  0.01
hi^2 (1-0.05)= 65618.1744483926
Test passed for alpha =  0.05
hi^2 (1-0.1)= 65487.158839407006
Test passed for alpha =  0.1
---- TEST 3 ------
m= 124999 r= 16
hi^2= 30.000480038379564
hi^2 (1-0.01)= 4028.4723381595445
Test passed for alpha =  0.01
hi^2 (1-0.05)= 3968.8659355964824
Test passed for alpha =  0.05
hi^2 (1-0.1)= 3937.089982981645
Test passed for alpha =  0.1
